## 🧬 CNN for TFBS Classification using k-mer Embeddings (Optuna-CNN-k-mer)

This notebook presents a deep learning pipeline for classifying DNA sequences as transcription factor binding sites (TFBS) or non-TFBS using k-mer based tokenization. DNA sequences are first broken into overlapping k-mers, which are then embedded via a trainable embedding layer. A dynamic CNN architecture is constructed, where the number of convolutional layers, filter sizes, number of filters, activation functions, and dropout rates are treated as tunable hyperparameters.

To automate and optimize the model architecture, the notebook employs Optuna, a powerful hyperparameter optimization framework. The objective function maximizes validation accuracy by searching across a defined hyperparameter space. The best-performing model is then saved for downstream inference.

Overall, this approach combines domain-aware sequence preprocessing with automated architecture search to yield a tailored CNN model capable of learning motif patterns in genomic sequences for TFBS prediction.


In [1]:
import pandas as pd
import numpy as np
import torch
import optuna

import sys

sys.path.append("../utils")

from initialize_results_df import initialize_results_df
from load_sequence_data import load_sequence_data
from optuna_cnn_kmer_utils import *
from k_mer_data_loader import *

In [3]:
k = 5
stride = 1
embedding_dim = 64
max_len = 96  # Could do 97: (101 - K + 1) // S = 97
batch_size = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab = build_kmer_vocab(k)
vocab_size = len(vocab) + 1
data_dir = "..\\Data"
excel_dir = "..\\Outputs\\optuna_cnn_kmer.xlsx"

results_df, excel_df = initialize_results_df(data_dir, excel_dir)

train_df = load_sequence_data(results_df["train_path"][0])
test_df = load_sequence_data(results_df["test_path"][0])

vocab = build_kmer_vocab(k)
vocab_size = len(vocab) + 1

train_loader, valid_loader, test_loader = prepare_kmer_loaders(
    train_df["sequence"].tolist(),
    train_df["label"].values,
    test_df["sequence"].tolist(),
    test_df["label"].values,
    vocab,
    k,
    stride,
    max_len,
    batch_size,
)

In [4]:
search_space = {
    "num_layers": {"type": "int", "low": 4, "high": 8},
    "embedding_dim": {"type": "categorical", "choices": [64]},
    "units": {"type": "categorical", "choices": [32, 64, 128]},
    "kernel_size": {"type": "categorical", "choices": [5, 7, 11]},
    "activation": {"type": "categorical", "choices": ["relu", "gelu", "silu"]},
    "dropout": {"type": "float", "low": 0.1, "high": 0.5},
}

best_model, best_params, acc, study = run_optuna_pipeline(
    train_loader,
    valid_loader,
    vocab_size=len(vocab) + 1,
    device="cuda",
    epochs=10,
    n_trials=15,
    max_len=96,
    save_path="../Models/best_model_kmer.pt",
    search_space=search_space,
)

print(best_params)
print(acc)

[I 2025-05-04 02:29:43,871] A new study created in memory with name: no-name-ba9d4926-3ab2-4fb6-9b24-d4ecf9cb182f
[I 2025-05-04 02:32:40,442] Trial 0 finished with value: 0.8894866704940796 and parameters: {'num_layers': 8, 'embedding_dim': 64, 'units_0': 64, 'kernel_size_0': 7, 'activation_0': 'silu', 'dropout_0': 0.4466458552647049, 'units_1': 128, 'kernel_size_1': 7, 'activation_1': 'relu', 'dropout_1': 0.37092693656724307, 'units_2': 32, 'kernel_size_2': 7, 'activation_2': 'silu', 'dropout_2': 0.48471193871923834, 'units_3': 64, 'kernel_size_3': 11, 'activation_3': 'relu', 'dropout_3': 0.41448445075946194, 'units_4': 128, 'kernel_size_4': 11, 'activation_4': 'gelu', 'dropout_4': 0.33867273953853816, 'units_5': 64, 'kernel_size_5': 7, 'activation_5': 'gelu', 'dropout_5': 0.17380016418959987, 'units_6': 32, 'kernel_size_6': 11, 'activation_6': 'gelu', 'dropout_6': 0.47288629320113895, 'units_7': 64, 'kernel_size_7': 5, 'activation_7': 'gelu', 'dropout_7': 0.3496816138430259}. Best is

{'num_layers': 6, 'embedding_dim': 64, 'units_0': 32, 'kernel_size_0': 11, 'activation_0': 'gelu', 'dropout_0': 0.4412236033491853, 'units_1': 128, 'kernel_size_1': 7, 'activation_1': 'relu', 'dropout_1': 0.15430686710722163, 'units_2': 64, 'kernel_size_2': 5, 'activation_2': 'gelu', 'dropout_2': 0.23495445825315997, 'units_3': 32, 'kernel_size_3': 7, 'activation_3': 'gelu', 'dropout_3': 0.13135844759781748, 'units_4': 32, 'kernel_size_4': 11, 'activation_4': 'relu', 'dropout_4': 0.18943200152157363, 'units_5': 64, 'kernel_size_5': 11, 'activation_5': 'gelu', 'dropout_5': 0.2398955681564483}
0.8898755311965942


In [5]:
study.best_params
import json

with open("../Models/final_model_hparams.json", "w") as f:
    json.dump(study.best_params, f)

In [7]:
# ✅ Load the saved best model weights
best_model.load_state_dict(torch.load("../Models/best_model_kmer.pt"))
best_model.to(device)
best_model.eval()

# ✅ Evaluate on test_loader
acc_test, preds_test, labels_test = evaluate(best_model, test_loader, device)

print(f"Test Accuracy: {acc_test:.4f}")

C:\Users\harsh\AppData\Local\Temp\ipykernel_22820\1109353383.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load("../Models/best_model_

Test Accuracy: 0.8878


## Looping through 50 folders

In [8]:
# Paths
data_dir = "../Data"
excel_path = "../Outputs/50_CNN_KM.xlsx"

# Load dataframes
results_df, excel_df = initialize_results_df(data_dir, excel_path)

Excel file saved at: ../Outputs/50_CNN_KM.xlsx


In [10]:
# load hp from JSON
with open("../Models/final_model_hparams.json", "r") as f:
    hp = json.load(f)

if "embedding_dim" not in hp:
    hp["embedding_dim"] = 64  # or whatever value you tuned

In [12]:
vocab = build_kmer_vocab(k=5)
vocab_size = len(vocab) + 1  # +1 for padding
max_len = 101  # or fixed, or largest length across all folders (up to you)

model = DynamicCNN(vocab_size, hp, max_len=max_len)
model.load_state_dict(torch.load("../Models/best_model_kmer.pt"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("✅ Model loaded and ready!")

✅ Model loaded and ready!


C:\Users\harsh\AppData\Local\Temp\ipykernel_22820\233208746.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../Models/best_model_kmer.p

In [13]:
for idx, row in results_df.iloc[:50].iterrows():
    train_path = row["train_path"]
    test_path = row["test_path"]
    folder_name = row["folder_name"]

    print(f"🔄 Processing {folder_name}")

    # --- Load data ---
    train_df = load_sequence_data(train_path)
    test_df = load_sequence_data(test_path)

    # --- Tokenize ---
    X_train = [
        tokenize_sequence(seq, vocab, k=5, stride=2)
        for seq in train_df["sequence"]
    ]
    X_test = [
        tokenize_sequence(seq, vocab, k=5, stride=2)
        for seq in test_df["sequence"]
    ]
    y_train = train_df["label"].tolist()
    y_test = test_df["label"].tolist()

    # --- Compute max_len dynamically (or set fixed if preferred) ---
    max_len = max(
        max(len(seq) for seq in X_train), max(len(seq) for seq in X_test)
    )

    # --- Prepare datasets/loaders ---
    train_dataset = PreTokenizedDataset(X_train, y_train, max_len=max_len)
    test_dataset = PreTokenizedDataset(X_test, y_test, max_len=max_len)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=32, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

    # --- Fine-tune same model ---
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss()
    train_one_epoch(model, train_loader, optimizer, criterion, device)

    # --- Evaluate ---
    train_acc, train_preds, train_labels = evaluate(
        model, train_loader, device
    )
    test_acc, test_preds, test_labels = evaluate(model, test_loader, device)

    # Optional: calculate PR AUC, ROC AUC
    from sklearn.metrics import average_precision_score, roc_auc_score

    train_probs = train_preds.numpy()
    test_probs = test_preds.numpy()

    train_pr_auc = average_precision_score(train_labels.numpy(), train_probs)
    train_roc_auc = roc_auc_score(train_labels.numpy(), train_probs)

    test_pr_auc = average_precision_score(test_labels.numpy(), test_probs)
    test_roc_auc = roc_auc_score(test_labels.numpy(), test_probs)

    # ✅ Log metrics
    excel_df.at[idx, "train_accuracy"] = train_acc
    excel_df.at[idx, "test_accuracy"] = test_acc
    excel_df.at[idx, "pr-roc"] = test_roc_auc
    excel_df.at[idx, "pr-auc"] = test_pr_auc

    print(
        f"✅ {folder_name}: train_acc={train_acc:.4f}, test_acc={test_acc:.4f}"
    )

excel_df.iloc[: idx + 1].to_excel(excel_path, index=False)
print(f"EXCEL SAVEd TO (excel_path)")

# # ✅ Save final model
torch.save(model.state_dict(), "../Models/Cnn_kmer_50.pt")
print("✅ Final model saved!")

🔄 Processing wgEncodeAwgTfbsBroadDnd41CtcfUniPk
✅ wgEncodeAwgTfbsBroadDnd41CtcfUniPk: train_acc=0.8460, test_acc=0.8268
🔄 Processing wgEncodeAwgTfbsBroadDnd41Ezh239875UniPk
✅ wgEncodeAwgTfbsBroadDnd41Ezh239875UniPk: train_acc=0.6474, test_acc=0.5529
🔄 Processing wgEncodeAwgTfbsBroadGm12878CtcfUniPk
✅ wgEncodeAwgTfbsBroadGm12878CtcfUniPk: train_acc=0.8736, test_acc=0.8411
🔄 Processing wgEncodeAwgTfbsBroadGm12878Ezh239875UniPk
✅ wgEncodeAwgTfbsBroadGm12878Ezh239875UniPk: train_acc=0.6510, test_acc=0.5855
🔄 Processing wgEncodeAwgTfbsBroadH1hescChd1a301218aUniPk
✅ wgEncodeAwgTfbsBroadH1hescChd1a301218aUniPk: train_acc=0.6321, test_acc=0.5988
🔄 Processing wgEncodeAwgTfbsBroadH1hescCtcfUniPk
✅ wgEncodeAwgTfbsBroadH1hescCtcfUniPk: train_acc=0.9014, test_acc=0.8788
🔄 Processing wgEncodeAwgTfbsBroadH1hescEzh239875UniPk
✅ wgEncodeAwgTfbsBroadH1hescEzh239875UniPk: train_acc=0.6536, test_acc=0.5891
🔄 Processing wgEncodeAwgTfbsBroadH1hescJarid1aab26049UniPk
✅ wgEncodeAwgTfbsBroadH1hescJarid1aab2604

## User input

In [2]:
import sys

sys.path.append("../utils")

from optuna_cnn_kmer_utils import (
    build_kmer_vocab,
    load_optuna_cnn_kmer_model,
    predict_optuna_cnn_kmer,
)

In [3]:
k = 5
stride = 1
max_len = 96
device = "cuda"

In [4]:
vocab = build_kmer_vocab(k=k)

model_path = "../Models/best_model_kmer.pt"
config_path = "../Models/final_model_hparams.json"

model, hp = load_optuna_cnn_kmer_model(
    model_path, config_path, vocab_size=len(vocab) + 1, device=device
)

c:\Users\harsh\Downloads\CAPSTONE-PROJECT\Notebooks\../utils\optuna_cnn_kmer_utils.py:263: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(mod

In [5]:
sequence = input("Enter a DNA sequence: ").strip()
label, confidence = predict_optuna_cnn_kmer(
    sequence, model, vocab, k=k, stride=stride, max_len=max_len, device=device
)

print(f"Prediction: {label} (Confidence: {confidence}%)")

Prediction: TFBS (Confidence: 99.98%)
